In [1]:
import numpy as np
from src.Utils.load_URM import load_URM
from src.Utils.cold_melting_warm_splitting import *

In [2]:
from src.Utils.load_ICM import load_ICM

ICM_all = load_ICM("in/data_ICM_title_abstract.csv")

ICM_all

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in COOrdinate format>

In [3]:
from src.Utils.ICM_preprocessing import binarize_ICM

binarize_ICM(ICM_all)

ICM_all.data

array([1, 1, 1, ..., 1, 1, 1])